In [6]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo


import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "random-string"


# Using sentence transformers all-MiniLM-L6-v2
embeddings = OpenAIEmbeddings(openai_api_base="http://localhost:8444/v1")

# Qdrant client
client = QdrantClient("localhost", port=6333)
qdrant_docsearch = Qdrant(client=client, collection_name="new_recipe_collection", embeddings=embeddings)

# chat completion llm
llm = ChatOpenAI(
    openai_api_base="http://localhost:8111/v1", max_tokens=128
)

# llm = ChatOpenAI(openai_api_key="the_api_key")


# Using OpenAI directly
# qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=qdrant_docsearch.as_retriever())

# Defining metadata fields
metadata_field_info = [
    AttributeInfo(
        name="recipe",
        description="The recipe of the dish",
        type="string",
    ),
    AttributeInfo(
        name="image",
        description="The file name of the image",
        type="string",
    ),
]
document_content_description = "Title of the recipe"

# Using SelfQueryRetriever
retriever = SelfQueryRetriever.from_llm(
    llm, qdrant_docsearch, document_content_description, metadata_field_info, verbose=True, enable_limit=True, 
)

# print(retriever.get_relevant_documents("any recipes with chicken and leek"))

# Using Vicuna via Premai app 
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

query = "what recipes do you have with chicken and leek that does not use an oven?"
result = qa({"query": query})

print(result)

query='chicken leek' filter=Operation(operator=<Operator.NOT: 'not'>, arguments=[Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='recipe', value='oven')]) limit=None
{'query': 'what recipes do you have with chicken and leek that does not use an oven?', 'result': 'Creamy Leek Soup and Lemon-Rhubarb Chicken can be made without using an oven.', 'source_documents': [Document(page_content='Beef Stew with Leeks', metadata={'image': 'beef-stew-with-leeks-356409.jpg', 'recipe': 'Place a large, heavy pot or Dutch oven over medium-high heat and add the blended oil. Season the beef aggressively with kosher salt and pepper. When the oil is hot, add the beef and sear on all sides, 5 to 6 minutes. Add all the chopped vegetables to the pan with the beef and sauté for 2 minutes. Add the tomato paste and stir for 1 minute. Deglaze the pan with the red wine and red wine vinegar, and let them reduce completely away.\nAdd 3 cups of the water, 2 teaspoons salt and a generous grinding of pe